<a href="https://colab.research.google.com/github/physalily/colab/blob/master/enigma_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from keras.layers     import Input, Dense, GRU, LSTM, RepeatVector, Dropout
from keras.models   import Model
from keras.layers.core import Flatten
from keras.callbacks    import LambdaCallback
from keras.optimizers  import SGD, RMSprop, Adam
from keras.layers.wrappers  import Bidirectional as Bi
from keras.layers.wrappers  import TimeDistributed as TD
from keras.layers     import merge
from keras.utils       import to_categorical

import glob
import random
import numpy as np
import tensorflow as tf

Using TensorFlow backend.


In [0]:
class enigma():
    def __init__(self, key1=0, key2=0):
        self.rotor1 = rotor(key1, rotorNo=1)
        self.rotor2 = rotor(key2, rotorNo=2)
        self.key1 = key1
        self.key2 = key2
        self.reflector = reflector()

    def print_rotor(self):
        print('rotor_print')
        
    def encryption(self, char):
        x = self.rotor1.current_in(char)
        x = self.rotor2.current_in(x)
        x = self.reflector.reflect(x)
        x = self.rotor2.reverse_in(x)
        x = self.rotor1.reverse_in(x)
        self.rotor1.indent()
        self.rotor2.indent()
        return x

    def statereset(self):
        self.rotor1 = rotor(self.key1, rotorNo=1)
        self.rotor2 = rotor(self.key2, rotorNo=2)


class reflector():
    def __init__(self):
        self.reflectorIn  = [chr(ord('a')+i) for i in range(26)]
        self.reflectorOut = [chr(ord('a')+i) for i in range(26)]
        randlist = [i for i in range(26)]
        random.seed(0)
        randlist = random.sample(randlist, 26)
        for i in range(13):
            self.outSwap(randlist[2*i], randlist[2*i+1])
            
    def reflect(self, char):
        return self.reflectorIn[self.reflectorOut.index(char)]

    def outSwap(self, indexA, indexB):
        tmp = self.reflectorOut[indexA]
        self.reflectorOut[indexA] = self.reflectorOut[indexB]
        self.reflectorOut[indexB] = tmp

class rotor():
    def __init__(self, key=0, rotorNo=0):
        random.seed(rotorNo)
        self.rotorIn  = [chr(ord('a')+i) for i in range(26)]
        self.rotorOut = random.sample([chr(ord('a')+i) for i in range(26)], 26)
        self.rotorNo = rotorNo
        self.state = 1
        for i in range(key):
            self.indent()
    
    def current_in(self, char):
        return self.rotorOut[self.rotorIn.index(char)]

    def reverse_in(self, char):
        return self.rotorIn[self.rotorOut.index(char)]

    def indent(self):
        self.state = self.state+1
        if (self.state%(10**(self.rotorNo-1)) == 0):
            self.rotorOut = self.rotorOut[1:26]+[self.rotorOut[0]]


In [0]:
all_data_raw = ''
train_data_raw = ''
train_data_cry = ''

Alphabet = [chr(ord('a')+i) for i in range(26)]

drive_corp_root = '/content/drive/My Drive/bbc/*/*'
eni = enigma()
for filename in glob.glob(drive_corp_root):
  try:
    text = open(filename).read()
  except Exception as ex:
    continue
  all_data_buffer = []
  raw_data_buffer = []
  cry_data_buffer = []

  for char in list(text.lower()):
    if char in Alphabet:
      all_data_buffer.append(char)
      raw_data_buffer.append(char)
      cry_data_buffer.append(eni.encryption(char))
    else:
      all_data_buffer.append(char)

  all_data_raw += ''.join(all_data_buffer)
  train_data_raw += ''.join(raw_data_buffer)
  train_data_cry += ''.join(cry_data_buffer)

In [0]:
train_data_raw = np.array([(ord(i)-ord('a')) for i in train_data_raw])
train_data_cry = np.array([(ord(i)-ord('a')) for i in train_data_cry])

In [0]:
one_hot_raw = to_categorical(train_data_raw[:3700000])
one_hot_cry = to_categorical(train_data_cry[:3700000])
one_hot_raw = one_hot_raw.reshape([-1, 100, 26])
one_hot_cry = one_hot_cry.reshape([-1, 100, 26])

In [0]:
timesteps = 100

inputs  = Input(shape=(timesteps, 26))
x         = Bi(GRU(512, dropout=0.10, recurrent_dropout=0.25, return_sequences=True))(inputs)
x         = TD(Dense(3000, activation='relu'))(x)
x         = Dropout(0.2)(x)
x         = TD(Dense(3000, activation='relu'))(x)
x         = Dropout(0.2)(x)
x         = TD(Dense(26, activation='softmax'))(x)

decript = Model(inputs, x)
decript.compile(optimizer=Adam(), loss='categorical_crossentropy')
decript.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 100, 26)           0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 100, 1024)         1655808   
_________________________________________________________________
time_distributed_7 (TimeDist (None, 100, 3000)         3075000   
_________________________________________________________________
dropout_5 (Dropout)          (None, 100, 3000)         0         
_________________________________________________________________
time_distributed_8 (TimeDist (None, 100, 3000)         9003000   
_________________________________________________________________
dropout_6 (Dropout)          (None, 100, 3000)         0         
_________________________________________________________________
time_distributed_9 (TimeDist (None, 100, 26)           7802

In [0]:
decript.fit(x=one_hot_cry, y=one_hot_raw,
            batch_size=300,
            epochs=35,
            validation_split=0.1)

Train on 33300 samples, validate on 3700 samples
Epoch 1/35
33300/33300 [==============================] - 132s 4ms/step - loss: 2.7147 - val_loss: 2.6049
Epoch 2/35
33300/33300 [==============================] - 132s 4ms/step - loss: 2.5804 - val_loss: 2.4622
Epoch 3/35
33300/33300 [==============================] - 131s 4ms/step - loss: 2.4752 - val_loss: 2.3565
Epoch 4/35
33300/33300 [==============================] - 131s 4ms/step - loss: 2.2108 - val_loss: 1.6193
Epoch 5/35
33300/33300 [==============================] - 131s 4ms/step - loss: 1.5072 - val_loss: 0.5661
Epoch 6/35
33300/33300 [==============================] - 130s 4ms/step - loss: 0.8085 - val_loss: 0.1223
Epoch 7/35
33300/33300 [==============================] - 130s 4ms/step - loss: 0.4075 - val_loss: 0.0341
Epoch 8/35
33300/33300 [==============================] - 130s 4ms/step - loss: 0.2270 - val_loss: 0.0146
Epoch 9/35
33300/33300 [==============================] - 130s 4ms/step - loss: 0.3309 - val_loss: 0.18

In [0]:
testA = [chr(ord('r')) for i in range(100)]
testA = ['h','e','l','l','o','w','o','r','l','d','h','e','l','l','o','h','e','l','l','o','h','e','l','l','o','h','e','l','l','o','h','e','l','l','o','h','e','l','l','o','h','e','l','l','o','h','e','l','l','o','h','e','l','l','o','h','e','l','l','o','h','e','l','l','o','h','e','l','l','o','h','e','l','l','o','h','e','l','l','o','h','e','l','l','o','h','e','l','l','o','h','e','l','l','o','h','e','l','l','o']
testEnigmaA = enigma()
print(len(testA))
testAcrypt = [testEnigmaA.encryption(test) for test in testA]
testEnigmaA.statereset()
testADecry = [testEnigmaA.encryption(test) for test in testA]
testpre = np.array([(ord(i)-ord('a')) for i in testAcrypt])
testpre = to_categorical(testpre)
testpre = testpre.reshape((1, 100, 26))
test = decript.predict(testpre)
test = np.argmax(test, axis=2)
print(test.shape)
test = test.reshape(100,)
test = [chr(ord('a')+i) for i in test]
test[:10]

100
(1, 100)


['h', 'e', 'l', 'l', 'o', 'w', 'o', 'r', 'l', 'd']

In [0]:
decript.save('/content/drive/My Drive/tf_model/enigma_rnn3')
decript.save_weights('/content/drive/My Drive/tf_model/enigma_rnn3_w.hdf5')

In [0]:
[x for x in 'hell world' if x.strip()]

['h', 'e', 'l', 'l', 'w', 'o', 'r', 'l', 'd']